# Anomalies

In [1]:
import jinja2
import os
import pandas as pd
#from scipy.optimize import minimize
import statsmodels.api as sm

#%pylab inline

c:\users\samth\miniconda3\envs\econ\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [187]:
F = pd.read_stata("C:/Data/Thesis/Factors.dta").set_index("date")

ff1993 = ["rm", "smb", "hml"]
c1997 = ["rm", "smb", "hml", "wml"]
ff2015 = ["rm", "smb", "hmlo", "rmw", "cma"]
ff2016a = ["rm", "smb", "hml", "RMWr", "cma"]
ff2016b = ["rm", "smb", "hml", "RMWc", "cma"]
bs2015 = ["rm", "smb", "HMLm", "wml", "RMWc"]

In [73]:
F[["rm", "smb", "hml", "HMLm", "RMWc", "wml", "cma"]].corr()

,rm,smb,hml,HMLm,RMWc,wml,cma
rm,1.000000,0.275998,-0.257824,-0.176170,-0.295320,-0.132263,-0.384005
smb,0.275998,1.000000,-0.081247,-0.086265,-0.296307,-0.024375,-0.101610
hml,-0.257824,-0.081247,1.000000,0.805267,-0.280033,-0.186647,0.691061
HMLm,-0.176170,-0.086265,0.805267,1.000000,-0.360100,-0.631604,0.527279
RMWc,-0.295320,-0.296307,-0.280033,-0.360100,1.000000,0.314495,-0.063024
wml,-0.132263,-0.024375,-0.186647,-0.631604,0.314495,1.000000,-0.012957
cma,-0.384005,-0.101610,0.691061,0.527279,-0.063024,-0.012957,1.000000


In [195]:
def GRS(R, F, factors, start="1963-07", end="2016-12"):
    """
    Returns the GRS statistic of Gibbons, Ross and Shanken (1989)
    """

    alphas = []
    errors = []

    for p in R.columns:
        model = sm.OLS(R[start:end][p] - F[start:end]["rf"],
                       sm.add_constant(F[start:end][factors]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        errors.append(fit.resid)

    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()

    K = F[factors].shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)

    r = F[factors].mean().as_matrix()

    f = F[factors].as_matrix()

    Vf = (f-r).transpose().dot(f-r) / (T - 1)

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f

    return (T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))

In [83]:
def tbl_content(LHS, factors, start, end, coef_names, K, j, precision=2, transpose_j=False):

    N = len(LHS.columns)
    A = {name: {"coef": [], "tval": [], "rows": []} for name in coef_names}

    # for each anomaly portfolio
    for lhsi, lhs in enumerate(LHS.columns):

        lhs = LHS[start:end][lhs] - F[start:end]["rf"]  # DataFrame
        rhs = sm.add_constant(F[start:end][factors])    # DataFrame

        # regress the "anomaly" on the factors
        model = sm.OLS(lhs, rhs)
        fit = model.fit()

        # get the coefficients and their t-values
        coef = [round(b, precision) for b in fit.params]
        tval = [round(t, precision) for t in fit.tvalues]

        # for each coefficient
        for namei, name in enumerate(coef_names):  # ordered
            A[name]["coef"].append(coef[namei])
            A[name]["tval"].append(tval[namei])

    for coef in A:

        A[coef]["coef"] = pd.np.array(A[coef]["coef"]).reshape(K, j, j)
        A[coef]["tval"] = pd.np.array(A[coef]["tval"]).reshape(K, j, j)

        if transpose_j:
            for k, x in enumerate(A[coef]["coef"]):
                A[coef]["coef"][k] = A[coef]["coef"][k].transpose()
                A[coef]["tval"][k] = A[coef]["tval"][k].transpose()

        A[coef]["coef"] = A[coef]["coef"].tolist()
        A[coef]["tval"] = A[coef]["tval"].tolist()

    for coef in A:
        for row in range(j):
            A[coef]["rows"].append([])
            for k in range(K):
                A[coef]["rows"][row].extend(A[coef]["coef"][k][row])
                A[coef]["rows"][row].extend(A[coef]["tval"][k][row])
    return A

In [5]:
path = "C:/Users/samth/Dropbox/Thesis/Tex/Tables/"

env = jinja2.Environment(
    block_start_string='-%', block_end_string='%-',
    variable_start_string='=%', variable_end_string='%=',
    loader=jinja2.FileSystemLoader(path)
)

# GRS

In [234]:
anomalies = [
    {"name": "Size-BM", "path": "C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"},
    {"name": "Size-Acc", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_AC_5x5.csv"},
    {"name": "Size-Beta", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.csv"},
    {"name": "Size-Inv", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.CSV"},
    {"name": "Size-NI", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_NI_5x5.csv"},
    {"name": "Size-OP", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_OP_5x5.CSV"},
    {"name": "Size-Prior", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV"},
    {"name": "Size-RVar", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_RESVAR_5x5.csv"},
    {"name": "Size-Var", "path": "C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.csv"},
    {"name": "Size-BM-Inv", "path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV"},
    {"name": "Size-BM-OP", "path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_OP_2x4x4.CSV"},
    {"name": "Size-OP-Inv", "path": "C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV"},
]

for anomaly in anomalies:
    #print(anomaly)
    anomaly["df"] = pd.read_csv(anomaly["path"])
    anomaly["df"] = anomaly["df"].iloc[:642, 1:]
    anomaly["df"].index = F.index

    anomaly["GRS"] = {
        "CAPM": "{:.3f}".format(GRS(anomaly["df"], F, ["rm",])),
        "FF1993": "{:.3f}".format(GRS(anomaly["df"], F, ff1993)),
        "C1997": "{:.3f}".format(GRS(anomaly["df"], F, c1997)),
        "FF2015": "{:.3f}".format(GRS(anomaly["df"], F, ff2015)),
        "FF2016a": "{:.3f}".format(GRS(anomaly["df"], F, ff2016a)),
        "FF2016b": "{:.3f}".format(GRS(anomaly["df"], F, ff2016b)),
        "BS2015b": "{:.3f}".format(GRS(anomaly["df"], F, ["rm", "smb", "hml", "wml", "RMWc"])),
        "BS2015": "{:.3f}".format(GRS(anomaly["df"], F, bs2015)),
    }

All, All_BM, All_Non_BM = {"name": "All"}, {"name": "All-BM"}, {"name": "All-Non-BM"}

for anomaly in anomalies:
    if not "df" in All:
        All["df"] = anomaly["df"]
    All["df"] = All["df"].merge(anomaly["df"], left_index=True, right_index=True)
    if "BM" in anomaly["name"]:
        if not "df" in All_BM:
            All_BM["df"] = anomaly["df"]
        All_BM["df"] = All_BM["df"].merge(anomaly["df"], left_index=True, right_index=True)
    if not "BM" in anomaly["name"]:
        if not "df" in All_Non_BM:
            All_Non_BM["df"] = anomaly["df"]
        All_Non_BM["df"] = All_Non_BM["df"].merge(anomaly["df"], left_index=True, right_index=True)
    
    anomaly.pop("df")

for anomaly in [All, All_BM, All_Non_BM]:
    anomaly["GRS"] = {
        #"CAPM": "{:.3f}".format(GRS(anomaly["df"], F, ["rm",])),
        #"FF1993": "{:.3f}".format(GRS(anomaly["df"], F, ff1993)),
        "C1997": "{:.3f}".format(GRS(anomaly["df"], F, c1997)),
        #"FF2015": "{:.3f}".format(GRS(anomaly["df"], F, ff2015)),
        #"FF2016a": "{:.3f}".format(GRS(anomaly["df"], F, ff2016a)),
        "FF2016b": "{:.3f}".format(GRS(anomaly["df"], F, ff2016b)),
        "BS2015b": "{:.3f}".format(GRS(anomaly["df"], F, ["rm", "smb", "hml", "wml", "RMWc"])),
        "BS2015": "{:.3f}".format(GRS(anomaly["df"], F, bs2015)),
        "BS2015b": "{:.3f}".format(GRS(anomaly["df"], F, ["rm", "smb", "hml", "wml", "RMWc"])),
        "BS2015c": "{:.3f}".format(GRS(anomaly["df"], F, ["rm", "smb", "HMLm", "RMWc"])),
    }

    anomaly.pop("df")

anomalies.extend([All_BM, All_Non_BM, All])

In [235]:
template = env.get_template("GRS_template.txt")

In [236]:
context = {
    "caption": r"GRS Statistic. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:GRS",
    "model_names": ["C1997", "FF2016b", "BS2015", "BS2015b", "BS2015c"],
    "anomalies": anomalies,
}

In [237]:
with open(os.path.join(path, "GRS_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

## 2x4x4

In [57]:
template = env.get_template("2x4x4_template.txt")

## Size-OP-Inv

In [179]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_OP_INV_2x4x4.CSV"), 2, 4
# a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

In [180]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'3.43, 2.44, 2.46, 1.88'

### FF2015

In [58]:
factors, coef_names = ff2015, ["a", "b", "s", "v", "p", "i"]

In [59]:
A = tbl_content(a, factors, "1963-07", "2013-12", coef_names, K, j, 2, True)

In [60]:
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "v", "p", "i"],
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-OP-Inv Portfolios with FF2015 Factors. \footnotesize{July, 1963 through December, 2013 (606 Months).}",
    "label": "tbl:32_Size_OP_Inv_FF2015"
}

In [61]:
with open(os.path.join(path, "32_Size_OP_Inv_FF2015_196307_201312.tex"), "w") as table:
    table.write(template.render(context))

### FF2016b

In [67]:
factors, coef_names = ff2016b, ["a", "b", "s", "v", "p", "i"]

In [68]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, True)

In [69]:
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "v", "p", "i"],
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-OP-Inv Portfolios with FF2016b Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:32_Size_OP_Inv_FF2016b"
}

In [70]:
with open(os.path.join(path, "32_Size_OP_Inv_FF2016b_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

### BS2015

In [62]:
factors, coef_names = bs2015, ["a", "b", "s", "v", "m", "p"]

In [63]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, True)

In [64]:
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "v", "m", "p"],
    "var_name": "OP",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "caption": r"Size-OP-Inv Portfolios with BS2015 Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:32_Size_OP_Inv_BS2015"
}

In [65]:
with open(os.path.join(path, "32_Size_OP_Inv_BS2015_196307_201612.tex"), "w") as table:
    table.write(template.render(context))

## Size-BM-OP

In [181]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_OP_2x4x4.CSV"), 2, 4
# a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

In [182]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'2.32, 1.38, 1.21, 1.53'

## Size-BM-Inv

In [183]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/32_Portfolios_ME_BEME_INV_2x4x4.CSV"), 2, 4
# a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

In [184]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'2.39, 1.59, 1.69, 2.17'

# 5x5

In [109]:
template, transpose = env.get_template("5x5_template.txt"), False

## Size-Beta

In [196]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_BETA_5x5.csv"), 1, 5
# a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

In [197]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'1.78, 2.22, 1.64, 1.10'

### BS2015

In [111]:
factors, coef_names = bs2015, ["a", "b", "s", "v", "m", "p"]

In [112]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [113]:
model = "BS2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "m", "p"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [114]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2015

In [119]:
factors, coef_names = ff2015, ["a", "b", "s", "v", "p", "i"]

In [120]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [121]:
model = "FF2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [122]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016a

In [115]:
factors, coef_names = ff2016a, ["a", "b", "s", "v", "p", "i"]

In [116]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [117]:
model = "FF2016a"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [118]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016b

In [105]:
factors, coef_names = ff2016b, ["a", "b", "s", "v", "p", "i"]

In [106]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [107]:
model = "FF2016b"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [108]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

## Size-Vol

In [125]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_VAR_5x5.csv"), 1, 5
# a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

In [126]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'5.16, 4.64, 4.82, 4.21'

### BS2015

In [127]:
factors, coef_names = bs2015, ["a", "b", "s", "v", "m", "p"]

In [128]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [129]:
model = "BS2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "m", "p"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [130]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2015

In [131]:
factors, coef_names = ff2015, ["a", "b", "s", "v", "p", "i"]

In [132]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [133]:
model = "FF2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [134]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016a

In [135]:
factors, coef_names = ff2016a, ["a", "b", "s", "v", "p", "i"]

In [136]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [137]:
model = "FF2016a"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [138]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016b

In [139]:
factors, coef_names = ff2016b, ["a", "b", "s", "v", "p", "i"]

In [140]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [141]:
model = "FF2016b"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [142]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

## Size-Prior

In [143]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_Prior_12_2.CSV"), 1, 5
# a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

In [144]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'4.44, 4.19, 4.22, 3.70'

### BS2015

In [145]:
factors, coef_names = bs2015, ["a", "b", "s", "v", "m", "p"]

In [146]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [147]:
model = "BS2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "m", "p"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [148]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2015

In [149]:
factors, coef_names = ff2015, ["a", "b", "s", "v", "p", "i"]

In [150]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [151]:
model = "FF2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [152]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016a

In [153]:
factors, coef_names = ff2016a, ["a", "b", "s", "v", "p", "i"]

In [154]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [155]:
model = "FF2016a"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [156]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016b

In [157]:
factors, coef_names = ff2016b, ["a", "b", "s", "v", "p", "i"]

In [158]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [159]:
model = "FF2016b"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [160]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

## Size-Inv

In [161]:
a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_ME_INV_5x5.CSV"), 1, 5
# a, K, j = pd.read_csv("C:/Data/FrenchDartmouth/25_Portfolios_5x5.CSV"), 1, 5
a = a.iloc[:642, 1:]
a.index = F.index

In [162]:
"{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(GRS(a, F, ff2015), GRS(a, F, ff2016a), GRS(a, F, ff2016b), GRS(a, F, bs2015))

'3.42, 2.44, 2.46, 1.97'

### BS2015

In [163]:
factors, coef_names = bs2015, ["a", "b", "s", "v", "m", "p"]

In [164]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [165]:
model = "BS2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "m", "p"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+m^iMom+p^iProf',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [166]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2015

In [167]:
factors, coef_names = ff2015, ["a", "b", "s", "v", "p", "i"]

In [168]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [169]:
model = "FF2015"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [170]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016a

In [171]:
factors, coef_names = ff2016a, ["a", "b", "s", "v", "p", "i"]

In [172]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [173]:
model = "FF2016a"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [174]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))

### FF2016b

In [175]:
factors, coef_names = ff2016b, ["a", "b", "s", "v", "p", "i"]

In [176]:
A = tbl_content(a, factors, "1963-07", "2016-12", coef_names, K, j, 2, transpose)

In [177]:
model = "FF2016b"
label = "{}_Size_Beta_{}".format(K*j*j, model)
context = {
    "A": A, "K": K, "j": j, "coef_names": ["a", "b", "s", "v", "p", "i"],
    "var_name": "Beta",
    "title": r'R^i=a^i+\beta^iMkt+s^iSize+v^iVal+p^iProf+i^iInv',
    "caption": r"Size-Beta Portfolios with "+model+r" Factors. \footnotesize{July, 1963 through December, 2016 (642 Months).}",
    "label": "tbl:{}".format(label)
}

In [178]:
with open(os.path.join(path, "{}_196307_201612.tex".format(label)), "w") as table:
    table.write(template.render(context))